In [1]:
from IPython.core.display import display, HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
import datetime
import mapping.layeredbasemap as lbm
import eqcatalog
from eqcatalog.macro import (aggregate_traditional_macro_info,
                             aggregate_online_macro_info, get_isoseismal_macro_info)

In [3]:
enq_type_selector = widgets.Dropdown(
    options=[('DYFI', 'dyfi'), ('Traditional', 'traditional'), ('Isoseismal', 'isoseismal')],
    value='traditional',
    description='Enq. type:',
    disabled=False,
)
#display(enq_type_selector)

In [4]:
eqid_selector = widgets.IntText(
    value=1306,
    description='Event ID:',
    disabled=False
)
#display(eqid_selector)

In [5]:
agg_by_selector = widgets.Dropdown(
    options=[('No aggregation', ''), ('Commune', 'id_com'), ('Main commune', 'id_main'),
             ('Grid (5km)', 'grid_5')],
    value='id_main',
    description='Aggregate by:',
    disabled=False,
)
#display(agg_by_selector)

In [6]:
color_gradient_selector = widgets.Dropdown(
    options=['discrete', 'continuous'],
    value='discrete',
    description='Color gradient:',
    disabled=False,
)
#display(color_gradient_selector)

In [7]:
cmap_selector = widgets.Dropdown(
    options=['rob', 'usgs'],
    value='rob',
    description='Color map:',
    disabled=False,
)
#display(cmap_selector)

In [8]:
lon_slider = widgets.FloatRangeSlider(
    value=[2.4, 6.6],
    min=-1.,
    max=9.,
    step=0.1,
    description='Lon range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [9]:
lat_slider = widgets.FloatRangeSlider(
    value=[49.4, 51.8],
    min=48.,
    max=53.,
    step=0.1,
    description='Lat range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [10]:
date_slider = widgets.IntRangeSlider(
    value=[1910, 2020],
    min=1350,
    max=2020,
    step=1,
    description='Year range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(date_slider)

In [11]:
start_date_picker = widgets.DatePicker(
    value=datetime.date(1985, 1, 1),
    description='Start date:',
    disabled=False
)
#display(start_date_picker)

In [12]:
end_date_picker = widgets.DatePicker(
    value = datetime.date.today(),
    description='End date:',
    disabled=False
)
#display(end_date_picker)

In [13]:
mag_slider = widgets.FloatRangeSlider(
    value=[2., 7.0],
    min=-1.,
    max=7.0,
    step=0.1,
    description='Magnitude:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
#display(mag_slider)

In [14]:
depth_slider = widgets.IntRangeSlider(
    value=[0, 50],
    min=0,
    max=50,
    step=1,
    description='Depth range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
#display(depth_slider)

In [15]:
etype_selector = widgets.SelectMultiple(
    options=[('Earthquake', 'ke'),
            ('Suspected earthquake', 'se'),
            ('Induced event', 'ki'),
            ('Suspected induced event', 'si'),
            ('Quarry blast', 'qb'),
            ('Suspected quarry blast', 'sqb'),
            ('Controlled explosion', 'cb'),
            ('Suspected controlled explosion', 'scb'),
            ('Mine explosion', 'km'),
            ('Suspected mine explosion', 'sm')],
    value=['ke'],
    #rows=10,
    description='Event type:',
    disabled=False
)
#display(etype_selector)

In [16]:
plot_type_selector = widgets.Dropdown(
    options=['basemap', 'folium'],
    value='folium',
    description='Plot type:',
    disabled=False,
)
#display(plot_type_selector)

In [17]:
def plot_macro_map(id_earth, enq_type, aggregate_by,
                   color_gradient, cmap,
                   lon_range, lat_range):
    ## Step 1: aggregate
    min_fiability = 80
    if enq_type == 'dyfi':
        min_replies = 3
        query_info = 'cii'
        filter_floors = (0, 4)
        agg_method = 'mean'
        fix_records = True
        include_other_felt = True
        include_heavy_appliance = False
        remove_outliers = (0, 100)
        macro_info = aggregate_online_macro_info(id_earth, min_replies=min_replies,
                                query_info=query_info, min_fiability=min_fiability,
                                filter_floors=filter_floors, aggregate_by=aggregate_by,
                                agg_method=agg_method, fix_records=fix_records,
                                include_other_felt=include_other_felt,
                                include_heavy_appliance=include_heavy_appliance,
                                remove_outliers=remove_outliers)
    elif enq_type == 'isoseismal' and aggregate_by in ('id_com', 'id_main'):
        main_communes = {'id_main': True, 'id_com': False}[aggregate_by]
        macro_info = get_isoseismal_macro_info(id_earth, main_communes=main_communes,
                                                as_points=True)
    else:
        Imin_or_max = 'max'
        agg_subcommunes = 'mean'
        if enq_type == 'traditional':
            data_type = ''
        else:
            data_type = enq_type
        macro_info = aggregate_traditional_macro_info(id_earth, data_type=data_type,
                                min_fiability=min_fiability, Imin_or_max=Imin_or_max,
                                aggregate_by=aggregate_by, agg_method=agg_subcommunes)

    ## Step 2: plot
    if macro_info:
        plot_info = 'intensity'
        colorbar_style = "default"
        region = lon_range + lat_range
        projection = "merc"
        graticule_interval = (2, 1)
        symbol_style = None
        line_style = 'default'
        thematic_num_replies = False
        interpolate_grid = {}
        plot_pie = {}
        title =''
        copyright = ''
        text = macro_info.get_proc_info_text()
        text_box = {'pos': 'bl', 'text': text}
        map = macro_info.plot_map(region=region, projection=projection,
                    graticule_interval=graticule_interval, plot_info=plot_info,
                    int_conversion='round', symbol_style=symbol_style,
                    line_style=line_style, thematic_num_replies=thematic_num_replies,
                    interpolate_grid=interpolate_grid, cmap=cmap,
                    color_gradient=color_gradient, admin_level='auto', admin_source='statbel',
                    colorbar_style=colorbar_style, radii=[], plot_pie=plot_pie,
                    title=title, fig_filespec=None, copyright=copyright,
                    text_box=text_box, dpi=90, verbose=True)
    else:
        print('No macroseismic information available!')

In [18]:
interactive_plot = interactive(plot_macro_map,
                               id_earth=eqid_selector,
                               enq_type=enq_type_selector,
                               aggregate_by=agg_by_selector,
                               color_gradient=color_gradient_selector,
                               cmap=cmap_selector,
                               lon_range=lon_slider, lat_range=lat_slider)
#output = interactive_plot.children[-1]
#output.layout.height = '350px'
interactive_plot

interactive(children=(IntText(value=1306, description='Event ID:'), Dropdown(description='Enq. type:', index=1…